In [1]:
# Standard scientific Python imports
import matplotlib.pyplot as plt

# Import datasets, classifiers and performance metrics
from sklearn import datasets, metrics, svm
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from itertools import product
import numpy as np

from sklearn.svm import SVC

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

ModuleNotFoundError: No module named 'torch'

# MNIST Dataset

## Data Loading

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

full_data = torch.cat([train_dataset.data, test_dataset.data], dim=0)
full_targets = torch.cat([train_dataset.targets, test_dataset.targets], dim=0)

MNIST_data = full_data.view(len(full_data), -1).numpy().astype('float32') / 255.0

MNIST = {
    "data": MNIST_data,
    "target": full_targets.numpy()
}

## Analysis

### (0) Base Case

#### Pre-analysis

In [ ]:
# standardization
X_MNIST = StandardScaler().fit_transform(MNIST["data"])
y_MNIST = MNIST["target"]

# Split into train and test
X_train_MNIST, X_test_MNIST, y_train_MNIST, y_test_MNIST = train_test_split(X_MNIST, y_MNIST, test_size=0.2, shuffle=True, random_state=42)

#### Analysis

Let's try as the Vanilla Base Case: Train on the entire dataset using uniform random shuffling for each epoch.

In [5]:
# Shuffle the training data with seed 42
X_shuffled, y_shuffled = shuffle(X_train_MNIST, y_train_MNIST, random_state=42)

difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST)

for percentage in difficulty_range:
    n_samples = int(percentage * len(X_shuffled))
    X_curr = X_shuffled[:n_samples]
    y_curr = y_shuffled[:n_samples]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST, test_pred))

# plot results
plt.plot(difficulty_range * 100, test_accuracies, label="Test accuracy")
plt.plot(difficulty_range * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("MNIST Dataset Base Case: Random Sampling (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()


KeyboardInterrupt



### (1) Curriculum Learning

#### Pre-analysis

As curriculum learning is based on giving samples in increasing difficulty level to the model, we first need to define a difficulty function. We will base ours on difference between the distance of each point to the line that goes through all the data points.

In [ ]:
def compute_MNIST_difficulty(X, y, centroids):
    dist = np.linalg.norm(X - centroids[y], axis=1)
    return dist

In [ ]:
percentages = np.linspace(0.75, 1, 5)

#### Analysis

In [ ]:
# compute class centroids to later apply the difficulty function
centroids = np.zeros((10, X_train_MNIST.shape[1]))
for i in range(10):
    centroids[i] = X_train_MNIST[y_train_MNIST == digit].mean(axis=0)

# normalize the difficulty to [0, 1]
MNIST_difficulty = compute_MNIST_difficulty(X_train_MNIST, y_train_MNIST, centroids)
norm_MNIST_difficulty = (MNIST_difficulty - MNIST_difficulty.min()) / (MNIST_difficulty.max() - MNIST_difficulty.min())

# sort by difficulty
sorted_indices = np.argsort(norm_MNIST_difficulty)
X_sorted = X_train_MNIST[sorted_indices]
y_sorted = y_train_MNIST[sorted_indices]

difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST)

for percentage in difficulty_range:
    n_samples = int(percentage * len(X_sorted))
    X_curr = X_sorted[:n_samples]
    y_curr = y_sorted[:n_samples]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST, test_pred))

# plot results
plt.plot(percentages * 100, test_accuracies, label="Test accuracy")
plt.plot(percentages * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Curriculum Learning on MNIST Dataset (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (2) Self-Paced Learning

In Self-Paced Learning, the model is supposed to:

• learn from easier samples first (based on current loss)

• adaptively expand its training set to include harder samples as it becomes more confident

We will use the same X_train, X_test, y_train and y_test computed in the curriculum learning phase.
To implement the SPL we will introduce a difficulty threshold to let the model choose how many samples of this difficulty it is ready to handle.

Let's determine the best combination of parameters.

In [ ]:
def run_spl(X_train, y_train, X_test, y_test, classes, 
            lambda_t_init, lambda_growth, init_size, max_iters=10, random_state=42):
    np.random.seed(random_state)

    # Init training subset
    init_idx = np.random.choice(len(X_train), size=init_size, replace=False)
    X_curr = X_train[init_idx]
    y_curr = y_train[init_idx]

    # Remaining pool
    mask = np.ones(len(X_train), dtype=bool)
    mask[init_idx] = False
    X_masked = X_train[mask]
    y_masked = y_train[mask]

    # Initial model
    clf = SGDClassifier(loss="log_loss", random_state=random_state)
    clf.partial_fit(X_curr, y_curr, classes=classes)

    lambda_t = lambda_t_init
    test_accuracies = []
    init_acc = None

    for it in range(1, max_iters + 1):
        probas = clf.predict_proba(X_masked)
        epsilon = 1e-12
        correct_class_probs = probas[np.arange(len(y_masked)), y_masked]
        sample_losses = -np.log(np.maximum(correct_class_probs, epsilon))
        
        filter_mask = sample_losses <= lambda_t
        if not np.any(filter_mask):
            break

        X_filtered = X_masked[filter_mask]
        y_filtered = y_masked[filter_mask]

        # Update train and pool
        X_curr = np.vstack([X_curr, X_filtered])
        y_curr = np.concatenate([y_curr, y_filtered])
        X_masked = X_masked[~filter_mask]
        y_masked = y_masked[~filter_mask]

        clf.partial_fit(X_filtered, y_filtered)

        test_acc = accuracy_score(y_test, clf.predict(X_test))
        test_accuracies.append(test_acc)

        if it == 1:
            init_acc = test_acc  # Store initial test accuracy

        lambda_t += lambda_growth

    # Improvement over the first iteration
    test_accuracies = np.array(test_accuracies)
    acc_improvement = np.max(test_accuracies - init_acc) if init_acc is not None else 0.0

    return acc_improvement, test_accuracies



# Grid search parameters
lambda_t_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
lambda_growth_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
init_sizes = [50, 100, 150, 200, 250, 300]

best_score = 0
best_params = None
results = []

for lambda_t_init, lambda_growth, init_size in product(lambda_t_values, lambda_growth_values, init_sizes):
    score, acc_list = run_spl(X_train_MNIST, y_train_MNIST, X_test_MNIST, y_test_MNIST,
                              classes=np.unique(y_train_MNIST),
                              lambda_t_init=lambda_t_init,
                              lambda_growth=lambda_growth,
                              init_size=init_size)
    
    results.append(((lambda_t_init, lambda_growth, init_size), score))
    
    if score > best_score:
        best_params = (lambda_t_init, lambda_growth, init_size)

# Print best combination
print(f"Best params: lambda_t={best_params[0]}, lambda_growth={best_params[1]}, init_size={best_params[2]}")

In [ ]:
classes = np.unique(y_train_MNIST)

# definition learning parameters
# lambda t : init threshold corresponding to the maximum loss (which then determines difficulty)
# lambda growth : incrementation at every step
# the model is then able to decide how many samples of such difficilty it can handle
max_iters = 10
lambda_t = 0.25
lambda_growth = 0.3

# init training on small random subset
init_size = 50
init_idx = np.random.choice(len(X_train_MNIST), size=init_size, replace=False)
X_curr = X_train_MNIST[init_idx]
y_curr = y_train_MNIST[init_idx]

# remaining pool
mask = np.ones(len(X_train_MNIST), dtype=bool)
mask[init_idx] = False
X_masked = X_train_MNIST[mask]
y_masked = y_train_MNIST[mask]

# initial fit
clf = SGDClassifier(loss="log_loss", random_state=42)
clf.partial_fit(X_curr, y_curr, classes=classes)

train_accuracies = []
test_accuracies = []

for it in range(1, max_iters + 1):
    # calculation probs & loss per sample
    probas = clf.predict_proba(X_masked)
    sample_losses = -np.log(probas[np.arange(len(y_masked)), y_masked])

    filter_mask = sample_losses <= lambda_t
    if not np.any(filter_mask):
        break

    X_filtered = X_masked[filter_mask]
    y_filtered = y_masked[filter_mask]

    X_curr = np.vstack([X_curr, X_filtered])
    y_curr = np.concatenate([y_curr, y_filtered])

    X_masked = X_masked[~filter_mask]
    y_masked = y_masked[~filter_mask]

    clf.partial_fit(X_filtered, y_filtered)

    train_acc = accuracy_score(y_curr, clf.predict(X_curr))
    test_acc = accuracy_score(y_test_MNIST, clf.predict(X_test_MNIST))
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

    lambda_t += lambda_growth

# tracé des courbes d'apprentissage
plt.plot(range(1, len(test_accuracies)+1), test_accuracies, label="Test accuracy")
plt.plot(range(1, len(train_accuracies)+1), train_accuracies, label="Train accuracy")
plt.xlabel("Itération SPL")
plt.ylabel("Accuracy")
plt.title("Self-Paced Learning on MNIST Dataset (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (3) Hard-Example Mining

Hard-Example Mining consists in feeding the model only hard examples. In our case, we will consider that a sample is difficult if its normalized difficulty is greater or equal than 0,75 (in other words the top 25%).

In [ ]:
sorted_indices = np.argsort(norm_MNIST_difficulty)
X_sorted = X_MNIST[sorted_indices]
y_sorted = y_MNIST[sorted_indices]

# Curriculum training: increase % of training data
percentages = np.linspace(0.8, 1, 4)
train_accuracies = []
test_accuracies = []

# test set
X_train_all, X_test_MNIST, y_train_all, y_test_MNIST = train_test_split(X_sorted, y_sorted, test_size=0.2, random_state=42)

for percentage in percentages:
    n_samples = int(percentage * len(X_train_all))
    X_train_MNIST = X_train_all[:n_samples]
    y_train_MNIST = y_train_all[:n_samples]
    
    clf = LogisticRegression()
    clf.fit(X_train_MNIST, y_train_MNIST)
    
    train_acc = accuracy_score(y_train_MNIST, clf.predict(X_train_MNIST))
    test_acc = accuracy_score(y_test_MNIST, clf.predict(X_test_MNIST))
    
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    
# Plot
plt.plot(percentages * 100, test_accuracies, label="Test accuracy")
plt.plot(percentages * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Hard-Example Mining on MNIST Dataset (Logistic Regression)")
plt.legend()
plt.grid(True)
plt.show()

### (4) Reverse Curriculum Learning

We are implementing **Reverse Curriculum Learning (RCL)** where the model starts learning from easier goals that are close to the target and gradually works backwards to more challenging starting states.

In [ ]:
# compute class centroids to later apply the difficulty function
centroids = np.zeros((10, X_train_MNIST.shape[1]))
for i in range(10):
    centroids[i] = X_train_MNIST[y_train_MNIST == i].mean(axis=0)

# normalize the difficulty to [0, 1]
MNIST_difficulty = compute_MNIST_difficulty(X_train_MNIST, y_train_MNIST, centroids)
norm_MNIST_difficulty = (MNIST_difficulty - MNIST_difficulty.min()) / (MNIST_difficulty.max() - MNIST_difficulty.min())

# sort by difficulty
sorted_indices = np.argsort(norm_MNIST_difficulty)[::-1] ## Reverse order for RCL
X_sorted = X_train_MNIST[sorted_indices]
y_sorted = y_train_MNIST[sorted_indices]

difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST)

for percentage in difficulty_range:
    n_samples = int(percentage * len(X_sorted))
    X_curr = X_sorted[:n_samples]
    y_curr = y_sorted[:n_samples]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST, test_pred))

# plot results
plt.plot(percentages * 100, test_accuracies, label="Test accuracy")
plt.plot(percentages * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Reverse Curriculum Learning on MNIST Dataset (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (5) Stratified Monte-Carlo Sampling

**Stratified Monte Carlo Sampling** is a variance reduction technique where the input space is divided into distinct strata (subregions), and samples are drawn from each stratum. This ensures more uniform coverage of the space compared to standard Monte Carlo sampling, leading to more accurate and stable estimates with fewer samples.

In [ ]:
# compute difficulty and normalize it
centroids = np.zeros((10, X_train_MNIST.shape[1]))
for i in range(10):
    centroids[i] = X_train_MNIST[y_train_MNIST == i].mean(axis=0)

MNIST_difficulty = compute_MNIST_difficulty(X_train_MNIST, y_train_MNIST, centroids)
norm_MNIST_difficulty = (MNIST_difficulty - MNIST_difficulty.min()) / (MNIST_difficulty.max() - MNIST_difficulty.min())

# stratify based on difficulty
n_strata = 10
strata_bounds = np.linspace(0, 1, n_strata + 1)
strata_indices = [np.where((norm_MNIST_difficulty >= strata_bounds[i]) & 
                           (norm_MNIST_difficulty < strata_bounds[i+1]))[0]
                  for i in range(n_strata)]

# training over growing percentages of the dataset
difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST)

for percentage in difficulty_range:
    n_total_samples = int(percentage * len(X_train_MNIST))
    n_per_stratum = n_total_samples // n_strata

    selected_indices = []
    for indices in strata_indices:
        if len(indices) == 0:
            continue
        stratum_sample = np.random.choice(indices, min(n_per_stratum, len(indices)), replace=False)
        selected_indices.extend(stratum_sample)

    X_curr = X_train_MNIST[selected_indices]
    y_curr = y_train_MNIST[selected_indices]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST, test_pred))

# plot results
plt.plot(percentages * 100, test_accuracies, label="Test accuracy")
plt.plot(percentages * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Stratified Monte Carlo Sampling on MNIST Dataset (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

# MNIST Dataset with Gaussian Noise

## Data Loading

In [ ]:
# pour digits c'était :
# digits = datasets.load_digits()
# n_samples = len(digits.images)
# digits_data = digits.images.reshape((n_samples, -1))

# réadapter de sorte à avoir
# MNIST_GN_data & MNIST_GN

## Analysis

### (0) Base Case

#### Pre-analysis

In [ ]:
# standardization
X_MNIST_GN = StandardScaler().fit_transform(MNIST_GN_data)
y_MNIST_GN = MNIST_GN.target

# Split into train and test
X_train_MNIST_GN, X_test_MNIST_GN, y_train_MNIST_GN, y_test_MNIST_GN = train_test_split(X_MNIST_GN, y_MNIST_GN, test_size=0.2, shuffle=True, random_state=42)

#### Analysis

In [ ]:
# Shuffle the training data with seed 42
X_shuffled, y_shuffled = shuffle(X_train_MNIST_GN, y_train_MNIST_GN, random_state=42)

difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST_GN)

for percentage in difficulty_range:
    n_samples = int(percentage * len(X_shuffled))
    X_curr = X_shuffled[:n_samples]
    y_curr = y_shuffled[:n_samples]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST_GN)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST_GN, test_pred))

# plot results
plt.plot(difficulty_range * 100, test_accuracies, label="Test accuracy")
plt.plot(difficulty_range * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("MNIST Dataset with Gaussian Noise Base Case: Random Sampling (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (1) Curriculum Learning

#### Pre-Analysis

In [ ]:
def compute_MNIST_difficulty(X, y, centroids):
    dist = np.linalg.norm(X - centroids[y], axis=1)
    return dist

In [ ]:
percentages = np.linspace(0.75, 1, 5)

#### Analysis

In [ ]:
# compute class centroids to later apply the difficulty function
centroids = np.zeros((10, X_train_MNIST_GN.shape[1]))
for i in range(10):
    centroids[i] = X_train_MNIST_GN[y_train_MNIST_GN == i].mean(axis=0)

# normalize the difficulty to [0, 1]
MNIST_difficulty = compute_MNIST_difficulty(X_train_MNIST_GN, y_train_MNIST_GN, centroids)
norm_MNIST_difficulty = (MNIST_difficulty - MNIST_difficulty.min()) / (MNIST_difficulty.max() - MNIST_difficulty.min())

# sort by difficulty
sorted_indices = np.argsort(norm_MNIST_difficulty)
X_sorted = X_train_MNIST_GN[sorted_indices]
y_sorted = y_train_MNIST_GN[sorted_indices]

difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST_GN)

for percentage in difficulty_range:
    n_samples = int(percentage * len(X_sorted))
    X_curr = X_sorted[:n_samples]
    y_curr = y_sorted[:n_samples]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST_GN)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST_GN, test_pred))

# Plot results
plt.plot(difficulty_range * 100, test_accuracies, label="Test accuracy")
plt.plot(difficulty_range * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Curriculum Learning on MNIST Dataset with Gaussian Noise (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (2) Self-Paced Learning

We will use the same X_train, X_test, y_train and y_test computed in the curriculum learning phase.
To implement the SPL we will introduce a difficulty threshold to let the model choose how many samples of this difficulty it is ready to handle.

Let's determine the best combination of parameters.

In [ ]:
# Grid search parameters
lambda_t_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
lambda_growth_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
init_sizes = [50, 100, 150, 200, 250, 300]

best_score = 0
best_params = None
results = []

for lambda_t_init, lambda_growth, init_size in product(lambda_t_values, lambda_growth_values, init_sizes):
    score, acc_list = run_spl(X_train_MNIST_GN, y_train_MNIST_GN, X_test_MNIST_GN, y_test_MNIST_GN,
                              classes=np.unique(y_train_MNIST_GN),
                              lambda_t_init=lambda_t_init,
                              lambda_growth=lambda_growth,
                              init_size=init_size)
    
    results.append(((lambda_t_init, lambda_growth, init_size), score))
    
    if score > best_score:
        best_params = (lambda_t_init, lambda_growth, init_size)

# Print best combination
print(f"Best params: lambda_t={best_params[0]}, lambda_growth={best_params[1]}, init_size={best_params[2]}")

In [ ]:
classes = np.unique(y_train_MNIST_GN)

# definition learning parameters
# lambda t : init threshold corresponding to the maximum loss (which then determines difficulty)
# lambda growth : incrementation at every step
# the model is then able to decide how many samples of such difficilty it can handle
max_iters = 10
lambda_t = 0.25
lambda_growth = 0.3

# init training on small random subset
init_size = 50
init_idx = np.random.choice(len(X_train_MNIST_GN), size=init_size, replace=False)
X_curr = X_train_MNIST_GN[init_idx]
y_curr = y_train_MNIST_GN[init_idx]

# remaining pool
mask = np.ones(len(X_train_MNIST_GN), dtype=bool)
mask[init_idx] = False
X_masked = X_train_MNIST_GN[mask]
y_masked = y_train_MNIST_GN[mask]

# initial fit
clf = SGDClassifier(loss="log_loss", random_state=42)
clf.partial_fit(X_curr, y_curr, classes=classes)

train_accuracies = []
test_accuracies = []

for it in range(1, max_iters + 1):
    # calculation probs & loss per sample
    probas = clf.predict_proba(X_masked)
    sample_losses = -np.log(probas[np.arange(len(y_masked)), y_masked])

    filter_mask = sample_losses <= lambda_t
    if not np.any(filter_mask):
        break

    X_filtered = X_masked[filter_mask]
    y_filtered = y_masked[filter_mask]

    X_curr = np.vstack([X_curr, X_filtered])
    y_curr = np.concatenate([y_curr, y_filtered])

    X_masked = X_masked[~filter_mask]
    y_masked = y_masked[~filter_mask]

    clf.partial_fit(X_filtered, y_filtered)

    train_acc = accuracy_score(y_curr, clf.predict(X_curr))
    test_acc = accuracy_score(y_test_MNIST_GN, clf.predict(X_test_MNIST_GN))
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

    lambda_t += lambda_growth

# tracé des courbes d'apprentissage
plt.plot(range(1, len(test_accuracies)+1), test_accuracies, label="Test accuracy")
plt.plot(range(1, len(train_accuracies)+1), train_accuracies, label="Train accuracy")
plt.xlabel("Itération SPL")
plt.ylabel("Accuracy")
plt.title("Self-Paced Learning on MNIST Dataset with Gaussian Noise (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (3) Hard-Example Mining

In [ ]:
sorted_indices = np.argsort(norm_MNIST_difficulty)
X_sorted = X_MNIST_GN[sorted_indices]
y_sorted = y_MNIST_GN[sorted_indices]

# Curriculum training: increase % of training data
percentages = np.linspace(0.8, 1, 4)
train_accuracies = []
test_accuracies = []

# test set
X_train_all, X_test_MNIST_GN, y_train_all, y_test_MNIST_GN = train_test_split(X_sorted, y_sorted, test_size=0.2, random_state=42)

for percentage in percentages:
    n_samples = int(percentage * len(X_train_all))
    X_train_MNIST_GN = X_train_all[:n_samples]
    y_train_MNIST_GN = y_train_all[:n_samples]
    
    clf = LogisticRegression()
    clf.fit(X_train_MNIST_GN, y_train_MNIST_GN)
    
    train_acc = accuracy_score(y_train_MNIST_GN, clf.predict(X_train_MNIST_GN))
    test_acc = accuracy_score(y_test_MNIST_GN, clf.predict(X_test_MNIST_GN))
    
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    
# Plot
plt.plot(percentages * 100, test_accuracies, label="Test accuracy")
plt.plot(percentages * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Hard-Example Mining on MNIST Dataset with Gaussian Noise (Logistic Regression)")
plt.legend()
plt.grid(True)
plt.show()

### (4) Reverse Curriculum Learning

In [ ]:
# compute class centroids to later apply the difficulty function
centroids = np.zeros((10, X_train_MNIST_GN.shape[1]))
for i in range(10):
    centroids[i] = X_train_MNIST_GN[y_train_MNIST_GN == i].mean(axis=0)

# normalize the difficulty to [0, 1]
MNIST_difficulty = compute_MNIST_difficulty(X_train_MNIST_GN, y_train_MNIST_GN, centroids)
norm_MNIST_difficulty = (MNIST_difficulty - MNIST_difficulty.min()) / (MNIST_difficulty.max() - MNIST_difficulty.min())

# sort by difficulty
sorted_indices = np.argsort(norm_MNIST_difficulty)[::-1] ## Reverse order for RCL
X_sorted = X_train_MNIST_GN[sorted_indices]
y_sorted = y_train_MNIST_GN[sorted_indices]

difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST_GN)

for percentage in difficulty_range:
    n_samples = int(percentage * len(X_sorted))
    X_curr = X_sorted[:n_samples]
    y_curr = y_sorted[:n_samples]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST_GN)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST_GN, test_pred))

# plot results
plt.plot(percentages * 100, test_accuracies, label="Test accuracy")
plt.plot(percentages * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Reverse Curriculum Learning on MNIST Dataset with Gaussian Noise (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (5) Stratified Monte-Carlo Sampling

In [ ]:
# compute difficulty and normalize it
centroids = np.zeros((10, X_train_MNIST_GN.shape[1]))
for i in range(10):
    centroids[i] = X_train_MNIST_GN[y_train_MNIST_GN == i].mean(axis=0)

MNIST_difficulty = compute_MNIST_difficulty(X_train_MNIST_GN, y_train_MNIST_GN, centroids)
norm_MNIST_difficulty = (MNIST_difficulty - MNIST_difficulty.min()) / (MNIST_difficulty.max() - MNIST_difficulty.min())

# stratify based on difficulty
n_strata = 10
strata_bounds = np.linspace(0, 1, n_strata + 1)
strata_indices = [np.where((norm_MNIST_difficulty >= strata_bounds[i]) & 
                           (norm_MNIST_difficulty < strata_bounds[i+1]))[0]
                  for i in range(n_strata)]

# training over growing percentages of the dataset
difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST_GN)

for percentage in difficulty_range:
    n_total_samples = int(percentage * len(X_train_MNIST_GN))
    n_per_stratum = n_total_samples // n_strata

    selected_indices = []
    for indices in strata_indices:
        if len(indices) == 0:
            continue
        stratum_sample = np.random.choice(indices, min(n_per_stratum, len(indices)), replace=False)
        selected_indices.extend(stratum_sample)

    X_curr = X_train_MNIST_GN[selected_indices]
    y_curr = y_train_MNIST_GN[selected_indices]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST_GN)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST_GN, test_pred))

# plot results
plt.plot(percentages * 100, test_accuracies, label="Test accuracy")
plt.plot(percentages * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Stratified Monte Carlo Sampling on MNIST Dataset with Gaussian Noise (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

# MNIST Dataset with Impulse Noise

## Data Loading

In [ ]:
# pour digits c'était :
# digits = datasets.load_digits()
# n_samples = len(digits.images)
# digits_data = digits.images.reshape((n_samples, -1))

# réadapter de sorte à avoir
# MNIST_IN_data & MNIST_IN

## Analysis

### (0) Base Case

#### Pre-analysis

In [ ]:
# standardization
X_MNIST_IN = StandardScaler().fit_transform(MNIST_IN_data)
y_MNIST_IN = MNIST_IN.target

# Split into train and test
X_train_MNIST_IN, X_test_MNIST_IN, y_train_MNIST_IN, y_test_MNIST_IN = train_test_split(X_MNIST_IN, y_MNIST_IN, test_size=0.2, shuffle=True, random_state=42)

#### Analysis

In [ ]:
# Shuffle the training data with seed 42
X_shuffled, y_shuffled = shuffle(X_train_MNIST_IN, y_train_MNIST_IN, random_state=42)

difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST_IN)

for percentage in difficulty_range:
    n_samples = int(percentage * len(X_shuffled))
    X_curr = X_shuffled[:n_samples]
    y_curr = y_shuffled[:n_samples]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST_IN)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST_IN, test_pred))

# plot results
plt.plot(difficulty_range * 100, test_accuracies, label="Test accuracy")
plt.plot(difficulty_range * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("MNIST Dataset with Impulse Noise Base Case: Random Sampling (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (1) Curriculum Learning

#### Pre-analysis

In [ ]:
def compute_MNIST_difficulty(X, y, centroids):
    dist = np.linalg.norm(X - centroids[y], axis=1)
    return dist

In [ ]:
percentages = np.linspace(0.75, 1, 5)

#### Analysis

In [ ]:
# compute class centroids to later apply the difficulty function
centroids = np.zeros((10, X_train_MNIST_IN.shape[1]))
for i in range(10):
    centroids[i] = X_train_MNIST_IN[y_train_MNIST_IN == i].mean(axis=0)

# normalize the difficulty to [0, 1]
MNIST_difficulty = compute_MNIST_difficulty(X_train_MNIST_IN, y_train_MNIST_IN, centroids)
norm_MNIST_difficulty = (MNIST_difficulty - MNIST_difficulty.min()) / (MNIST_difficulty.max() - MNIST_difficulty.min())

# sort by difficulty
sorted_indices = np.argsort(norm_MNIST_difficulty)
X_sorted = X_train_MNIST_IN[sorted_indices]
y_sorted = y_train_MNIST_IN[sorted_indices]

difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST_IN)

for percentage in difficulty_range:
    n_samples = int(percentage * len(X_sorted))
    X_curr = X_sorted[:n_samples]
    y_curr = y_sorted[:n_samples]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST_IN)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST_IN, test_pred))

# Plot results
plt.plot(difficulty_range * 100, test_accuracies, label="Test accuracy")
plt.plot(difficulty_range * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Curriculum Learning on MNIST Dataset with Impulse Noise (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (2) Self-Paced Learning

We will use the same X_train, X_test, y_train and y_test computed in the curriculum learning phase.
To implement the SPL we will introduce a difficulty threshold to let the model choose how many samples of this difficulty it is ready to handle.

Let's determine the best combination of parameters.

In [ ]:
# Grid search parameters
lambda_t_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
lambda_growth_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
init_sizes = [50, 100, 150, 200, 250, 300]

best_score = 0
best_params = None
results = []

for lambda_t_init, lambda_growth, init_size in product(lambda_t_values, lambda_growth_values, init_sizes):
    score, acc_list = run_spl(X_train_MNIST_IN, y_train_MNIST_IN, X_test_MNIST_IN, y_test_MNIST_IN,
                              classes=np.unique(y_train_MNIST_IN),
                              lambda_t_init=lambda_t_init,
                              lambda_growth=lambda_growth,
                              init_size=init_size)
    
    results.append(((lambda_t_init, lambda_growth, init_size), score))
    
    if score > best_score:
        best_params = (lambda_t_init, lambda_growth, init_size)

# Print best combination
print(f"Best params: lambda_t={best_params[0]}, lambda_growth={best_params[1]}, init_size={best_params[2]}")

In [ ]:
classes = np.unique(y_train_MNIST_IN)

# definition learning parameters
# lambda t : init threshold corresponding to the maximum loss (which then determines difficulty)
# lambda growth : incrementation at every step
# the model is then able to decide how many samples of such difficilty it can handle
max_iters = 10
lambda_t = 0.25
lambda_growth = 0.3

# init training on small random subset
init_size = 50
init_idx = np.random.choice(len(X_train_MNIST_IN), size=init_size, replace=False)
X_curr = X_train_MNIST_IN[init_idx]
y_curr = y_train_MNIST_IN[init_idx]

# remaining pool
mask = np.ones(len(X_train_MNIST_IN), dtype=bool)
mask[init_idx] = False
X_masked = X_train_MNIST_IN[mask]
y_masked = y_train_MNIST_IN[mask]

# initial fit
clf = SGDClassifier(loss="log_loss", random_state=42)
clf.partial_fit(X_curr, y_curr, classes=classes)

train_accuracies = []
test_accuracies = []

for it in range(1, max_iters + 1):
    # calculation probs & loss per sample
    probas = clf.predict_proba(X_masked)
    sample_losses = -np.log(probas[np.arange(len(y_masked)), y_masked])

    filter_mask = sample_losses <= lambda_t
    if not np.any(filter_mask):
        break

    X_filtered = X_masked[filter_mask]
    y_filtered = y_masked[filter_mask]

    X_curr = np.vstack([X_curr, X_filtered])
    y_curr = np.concatenate([y_curr, y_filtered])

    X_masked = X_masked[~filter_mask]
    y_masked = y_masked[~filter_mask]

    clf.partial_fit(X_filtered, y_filtered)

    train_acc = accuracy_score(y_curr, clf.predict(X_curr))
    test_acc = accuracy_score(y_test_MNIST_IN, clf.predict(X_test_MNIST_IN))
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

    lambda_t += lambda_growth

# tracé des courbes d'apprentissage
plt.plot(range(1, len(test_accuracies)+1), test_accuracies, label="Test accuracy")
plt.plot(range(1, len(train_accuracies)+1), train_accuracies, label="Train accuracy")
plt.xlabel("Itération SPL")
plt.ylabel("Accuracy")
plt.title("Self-Paced Learning on MNIST Dataset with Impulsive Noise (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (3) Hard-Example Mining

In [ ]:
sorted_indices = np.argsort(norm_MNIST_difficulty)
X_sorted = X_MNIST_IN[sorted_indices]
y_sorted = y_MNIST_IN[sorted_indices]

# Curriculum training: increase % of training data
percentages = np.linspace(0.8, 1, 4)
train_accuracies = []
test_accuracies = []

# test set
X_train_all, X_test_MNIST_IN, y_train_all, y_test_MNIST_IN = train_test_split(X_sorted, y_sorted, test_size=0.2, random_state=42)

for percentage in percentages:
    n_samples = int(percentage * len(X_train_all))
    X_train_MNIST_IN = X_train_all[:n_samples]
    y_train_MNIST_IN = y_train_all[:n_samples]
    
    clf = LogisticRegression()
    clf.fit(X_train_MNIST_IN, y_train_MNIST_IN)
    
    train_acc = accuracy_score(y_train_MNIST_IN, clf.predict(X_train_MNIST_IN))
    test_acc = accuracy_score(y_test_MNIST_IN, clf.predict(X_test_MNIST_IN))
    
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    
# Plot
plt.plot(percentages * 100, test_accuracies, label="Test accuracy")
plt.plot(percentages * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Hard-Example Mining on MNIST Dataset with Impulsive Noise (Logistic Regression)")
plt.legend()
plt.grid(True)
plt.show()

### (4) Reverse Curriculum Learning

In [ ]:
# compute class centroids to later apply the difficulty function
centroids = np.zeros((10, X_train_MNIST_IN.shape[1]))
for i in range(10):
    centroids[i] = X_train_MNIST_IN[y_train_MNIST_IN == i].mean(axis=0)

# normalize the difficulty to [0, 1]
MNIST_difficulty = compute_MNIST_difficulty(X_train_MNIST_IN, y_train_MNIST_IN, centroids)
norm_MNIST_difficulty = (MNIST_difficulty - MNIST_difficulty.min()) / (MNIST_difficulty.max() - MNIST_difficulty.min())

# sort by difficulty
sorted_indices = np.argsort(norm_MNIST_difficulty)[::-1] ## Reverse order for RCL
X_sorted = X_train_MNIST_IN[sorted_indices]
y_sorted = y_train_MNIST_IN[sorted_indices]

difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST_IN)

for percentage in difficulty_range:
    n_samples = int(percentage * len(X_sorted))
    X_curr = X_sorted[:n_samples]
    y_curr = y_sorted[:n_samples]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST_IN)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST_IN, test_pred))

# plot results
plt.plot(percentages * 100, test_accuracies, label="Test accuracy")
plt.plot(percentages * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Reverse Curriculum Learning on MNIST Dataset with Impulsive Noise (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()

### (5) Stratified Monte-Carlo Sampling

In [ ]:
# compute difficulty and normalize it
centroids = np.zeros((10, X_train_MNIST_IN.shape[1]))
for i in range(10):
    centroids[i] = X_train_MNIST_IN[y_train_MNIST_IN == i].mean(axis=0)

MNIST_difficulty = compute_MNIST_difficulty(X_train_MNIST_IN, y_train_MNIST_IN, centroids)
norm_MNIST_difficulty = (MNIST_difficulty - MNIST_difficulty.min()) / (MNIST_difficulty.max() - MNIST_difficulty.min())

# stratify based on difficulty
n_strata = 10
strata_bounds = np.linspace(0, 1, n_strata + 1)
strata_indices = [np.where((norm_MNIST_difficulty >= strata_bounds[i]) & 
                           (norm_MNIST_difficulty < strata_bounds[i+1]))[0]
                  for i in range(n_strata)]

# training over growing percentages of the dataset
difficulty_range = np.linspace(0.1, 1.0, 10)
train_accuracies = []
test_accuracies = []

clf = SGDClassifier(random_state=42)
classes = np.unique(y_train_MNIST_IN)

for percentage in difficulty_range:
    n_total_samples = int(percentage * len(X_train_MNIST_IN))
    n_per_stratum = n_total_samples // n_strata

    selected_indices = []
    for indices in strata_indices:
        if len(indices) == 0:
            continue
        stratum_sample = np.random.choice(indices, min(n_per_stratum, len(indices)), replace=False)
        selected_indices.extend(stratum_sample)

    X_curr = X_train_MNIST_IN[selected_indices]
    y_curr = y_train_MNIST_IN[selected_indices]

    clf.partial_fit(X_curr, y_curr, classes=classes)

    train_pred = clf.predict(X_curr)
    test_pred = clf.predict(X_test_MNIST_IN)

    train_accuracies.append(accuracy_score(y_curr, train_pred))
    test_accuracies.append(accuracy_score(y_test_MNIST_IN, test_pred))

# plot results
plt.plot(percentages * 100, test_accuracies, label="Test accuracy")
plt.plot(percentages * 100, train_accuracies, label="Train accuracy")
plt.xlabel("Percentage of training data (by difficulty)")
plt.ylabel("Accuracy")
plt.title("Stratified Monte Carlo Sampling on MNIST Dataset with Impulsive Noise (SGDClassifier)")
plt.legend()
plt.grid(True)
plt.show()